# Part 3: Single-View Geometry

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
from sympy import *
from PIL import Image

# Provided functions

In [2]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [3]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [4]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [5]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    s_m = lines.dot(lines.T)
    w, v = np.linalg.eig(s_m)
    _min = np.argmin(w)
    vp = v[:, _min]
    vp = vp / vp[-1]
    return vp

In [6]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    pt1 = vpts[:, 0]
    pt2 = vpts[:, 1]
    horizon_line = np.cross(pt1, pt2)
    # Normalize.
    scale = np.sqrt(horizon_line[0]**2 + horizon_line[1]**2)
    horizon_line = horizon_line / scale
    return horizon_line

In [7]:
def plot_horizon_line(line, img):
    """
    Plots the horizon line.
    """
    x = np.arange(img.shape[1])
    y = (- line[2] - line[0] * x) / line[1]
    plt.figure()
    plt.imshow(im)
    plt.plot(x, y, 'r', linestyle='-', linewidth=3)
    plt.show()

In [8]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    vp1 = vpts[:, 0].reshape(3,1)
    vp2 = vpts[:, 1].reshape(3,1)
    vp3 = vpts[:, 2].reshape(3,1)
    
    f, px, py = symbols('f, px. py')
    K_i_T = Matrix([[1/f, 0, 0], [0, 1/f, 0], [-px/f, -py/f, 1]])
    K_i = Matrix([[1/f, 0, -px/f], [0, 1/f, -py/f], [0, 0, 1]])
    
    eq1 = Eq((vp1.T @ K_i_T @ K_i @ vp2)[0], 0)
    eq2 = Eq((vp1.T @ K_i_T @ K_i @ vp3)[0], 0)
    eq3 = Eq((vp2.T @ K_i_T @ K_i @ vp3)[0], 0)
    
    f, u, v = solve((eq1,eq2,eq3), (f,px,py))[0]
    
    return f, u, v

In [9]:
def get_rotation_matrix(vpts, f, u, v):
    """
    Computes the rotation matrix using the camera parameters.
    """
    Z = vpts[:, 0].reshape(3,1)
    X = vpts[:, 1].reshape(3,1)
    Y = vpts[:, 2].reshape(3,1)
    K = np.array([[f, 0, u], [0, f, v], [0, 0, 1]], dtype=float)
    K_inv = np.linalg.inv(K)
    
    r1 = K_inv.dot(X)
    r2 = K_inv.dot(Y)
    r3 = K_inv.dot(Z)
    
    r1 = r1 / np.linalg.norm(r1)
    r2 = r2 / np.linalg.norm(r2)
    r3 = r3 / np.linalg.norm(r3)
    
    R = np.concatenate((r1, r2, r3), axis=1)
    return R

In [10]:
def estimate_height(im,person, obj, vpts, horizon_line, ref_height):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    vp1 = vpts[:, 0].reshape(3,1)
    vp2 = vpts[:, 1].reshape(3,1)
    vp3 = vpts[:, 2].reshape(3,1)
    #可视化地平线
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.imshow(im)
    ax.plot(vp1[0],vp1[1],'+r')
    ax.plot(vp2[0],vp2[1],'+r')
    ax.plot([vp1[0], vp2[0]], [vp1[1], vp2[1]], 'g')
    #可视化人和物
    ax.plot([person[0][0], person[0][1]], [person[1][0], person[1][1]], 'g')
    ax.plot([obj[0][0], obj[0][1]], [obj[1][0], obj[1][1]], 'y')
    b0 = np.array([person[0][0], person[1][0], 1])
    t0 = np.array([person[0][1], person[1][1], 1])
    b = np.array([obj[0][0], obj[1][0], 1])#bt可能反了
    r = np.array([obj[0][1], obj[1][1], 1])
    #计算目标高度
    img_height_tar = abs(obj[1][0] - obj[1][1])
    #求bottom_line
    bottom_line = np.real(np.cross(b0.T, b.T))
    v = np.real(np.cross(bottom_line, horizon_line))#这里可能有问题horizonline
    v = v / v[2]
    #求top_line
    top_line = np.real(np.cross(v.T, t0.T))
    vertical_line = np.real(np.cross(r.T, b.T));
    t = np.real(np.cross(top_line.T, vertical_line.T))#这里可能有问题horizonline
    t = t / t[2]
    ax.plot([v[0], b0[0]], [v[1], b0[1]], 'm')
    ax.plot([v[0], t0[0]], [v[1], t0[1]], 'm')
    ax.plot([v[0], t[0]], [v[1], t[1]], 'm')
    ax.plot([b[0], t[0]], [b[1], t[1]], 'c')
    ax.plot([b[0], v[0]], [b[1], v[1]], 'm')
    #计算高度差
    left_up = np.linalg.norm(t - b) * np.linalg.norm(vp3.T - r)
    left_down = np.linalg.norm(r - b) * np.linalg.norm(vp3.T - t)
    H = ref_height
    height_tar = (H * left_down) / left_up
    plt.show()
    return height_tar

# Main function

In [11]:
im = np.asarray(Image.open('E:/UIUC_course/FALL_SEMESTER/Computer_Vision/MP4/MP4_part2_data/MP4_part2_data/CSL.jpg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    print("lines: ", lines)
    print("n:", n)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(lines)
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])

# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
# <YOUR IMPLEMENTATION> Plot the ground horizon line
plot_horizon_line(horizon_line,im)

Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
lines:  [[-2.60276108e+01 -4.07098528e+01  1.40148674e+01]
 [-4.54482127e+02 -5.01198352e+02 -4.15774398e+02]
 [ 9.11341588e+04  9.61216013e+04  9.16597040e+04]]
n: 3
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use th

In [12]:
print("horizon_line: ", horizon_line)
print("vpts: ", vpts)
print("a: ", vpts[:,0][:,np.newaxis].shape)
print("c: ", vpts[:,0][:,np.newaxis])
print("b: ", vpts[:,0].reshape(3,1))

horizon_line:  [-1.00467722e-02  9.99949530e-01 -2.15219016e+02]
vpts:  [[-2.45010665e+02  1.32478877e+03  4.50809524e+02]
 [ 2.12768188e+02  2.28540401e+02  7.67276174e+03]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00]]
a:  (3, 1)
c:  [[-245.01066458]
 [ 212.76818803]
 [   1.        ]]
b:  [[-245.01066458]
 [ 212.76818803]
 [   1.        ]]


In [13]:
(horizon_line[0])**2 + (horizon_line[1])**2

1.0

In [14]:
# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f,u,v = get_camera_parameters(vpts)
print(f,u,v)

-780.447080083736 524.854039253718 303.153165312346


In [15]:
# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix(vpts, f, u, v)
print(R)

[[-0.71418181  0.00999091  0.69988893]
 [ 0.06661428 -0.99438961  0.08216956]
 [ 0.69678323  0.10530661  0.70950944]]


In [17]:
# Part 4
# Record image coordinates for each object and store in map
im = np.asarray(Image.open('E:/UIUC_course/FALL_SEMESTER/Computer_Vision/MP4/MP4_part2_data/MP4_part2_data/CSL.jpg'))
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

print(coords)

Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of the spike statue
Click on the bottom coordinate of the spike statue
Click on the top coordinate of the lamp posts
Click on the bottom coordinate of the lamp posts
{'person': array([[626.955812  , 624.9536881 ],
       [466.68894009, 510.06829142],
       [  1.        ,   1.        ]]), 'CSL building': array([[471.45752185, 470.12277258],
       [152.3554866 , 303.1821543 ],
       [  1.        ,   1.        ]]), 'the spike statue': array([[602.26295048, 598.9260773 ],
       [193.06533939, 474.03006109],
       [  1.        ,   1.        ]]), 'the lamp posts': array([[711.71239076, 705.70601904],
       [325.87289192, 402.62097505],
       [  1.        ,   1.        ]])}


In [18]:
# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
#     5 feet, 6 inches = 167.64cm
#     6 feet = 182.88cm
    height = estimate_height(im, coords['person'], coords[obj], vpts, horizon_line, ref_height = 182.88)
    print(height)

Estimating height of CSL building
2165.715775094404
Estimating height of the spike statue
7538.23264425335
Estimating height of the lamp posts
719.9196305093795


In [19]:
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
#     5 feet, 6 inches = 167.64cm
#     6 feet = 182.88cm
    height = estimate_height(im, coords['person'], coords[obj], vpts, horizon_line, ref_height = 167.64)
    print(height)

Estimating height of CSL building
1985.2394605032034
Estimating height of the spike statue
6910.046590565572
Estimating height of the lamp posts
659.9263279669311
